The objective is to create a first model to use as a baseline for subsequent models and optimizations

In [72]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [74]:
pd.options.display.max_columns = None

In [89]:
df = pd.read_csv('../data/curated/features_rounded_news_USD_pair_EURUSD_2007_2018.csv')

In [125]:
df_high = df[df['impact'] == 'High'].reset_index()

In [126]:
len(df_high)

3256

In [127]:
df_high = df_high[columns_of_interest]

In [128]:
df_high.head(2)

,new,datetime,forecast_error,previous_error,forecast_error_ratio,previous_error_ratio,total_error_ratio,forecast_error_ratio_zscore,total_error_ratio_zscore,year,...,close_45_before,close_60_before,close_90_before,close_120_before,close_180_before,close_240_before,close_1140_before,close_2280_before,pips_agg_30_after,direction_agg_30_after
0,Trade Balance,2007-01-10 08:30:00,better,accurate,3.00,0.00,3.00,0.000000,0.000000,2007,...,12988,12993,12996,12992,12993,12984,13009,13028,12,down
1,Trade Balance,2007-02-13 08:30:00,worse,accurate,-2.86,-0.17,-3.03,-0.707107,-0.707107,2007,...,13012,13013,13004,13000,12999,12990,12960,13019,6,up


Encode categorical values as follows:

1. forecast_error and previous_error will be encoded using one hot encoding
2. fit transform for the new name


In [97]:
le = preprocessing.LabelEncoder()

df_high['new_id'] = le.fit_transform(df_high['new'])
dummy_forecast_error = pd.get_dummies(df_high['forecast_error'])
dummy_previous_error = pd.get_dummies(df_high['previous_error'])


In [98]:
dummy_forecast_error.columns

Index(['accurate', 'better', 'worse'], dtype='object')

In [99]:
dummy_previous_error.columns

Index(['accurate', 'better', 'worse'], dtype='object')

In [100]:

dummy_forecast_error.columns = ['fe_accurate', 'fe_better', 'fe_worse']
dummy_previous_error.columns = ['pe_accurate', 'pe_better', 'pe_worse']


In [101]:
dummy_forecast_error.head(5)

,fe_accurate,fe_better,fe_worse
0,0,1,0
1,0,0,1
2,1,0,0
3,0,1,0
4,0,0,1


In [102]:
df_high = pd.concat([df_high, dummy_forecast_error, dummy_previous_error], axis=1)

In [103]:
df_high.head(3)

,forecast_error,new,previous_error,week,forecast_error_ratio,year,quarter,month,weekday,close_released,...,total_error_ratio_zscore,datetime,direction_agg_30,new_id,fe_accurate,fe_better,fe_worse,pe_accurate,pe_better,pe_worse
0,better,Trade Balance,accurate,2,3.00,2007,1,1,2,12984,...,0.000000,2007-01-10 08:30:00,down,31,0,1,0,1,0,0
1,worse,Trade Balance,accurate,7,-2.86,2007,1,2,1,13020,...,-0.707107,2007-02-13 08:30:00,up,31,0,0,1,1,0,0
2,accurate,Trade Balance,accurate,10,-0.67,2007,1,3,4,13155,...,-0.036466,2007-03-09 08:30:00,down,31,1,0,0,1,0,0


We can now delete categorical values

In [105]:
df['direction_agg_30'] = np.where(df['direction_agg_30'] == 'up', 1, 0)

In [106]:
df_high.columns

Index(['forecast_error', 'new', 'previous_error', 'week',
       'forecast_error_ratio', 'year', 'quarter', 'month', 'weekday',
       'close_released', 'previous_error_ratio', 'total_error_ratio',
       'forecast_error_ratio_zscore', 'total_error_ratio_zscore', 'datetime',
       'direction_agg_30', 'new_id', 'fe_accurate', 'fe_better', 'fe_worse',
       'pe_accurate', 'pe_better', 'pe_worse'],
      dtype='object')

In [68]:
df_high = df_high.drop(['forecast_error', 'new', 'previous_error'], axis=1)

In [69]:
df_high.describe()

,week,forecast_error_ratio,year,quarter,month,weekday,close_released,previous_error_ratio,total_error_ratio,forecast_error_ratio_zscore,total_error_ratio_zscore,new_id,fe_accurate,fe_better,fe_worse,pe_accurate,pe_better,pe_worse,dir_agg_30_down,dir_agg_30_up
count,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000,3256.000000
mean,26.170455,-11.986198,2012.528563,2.475430,6.423526,2.531634,12839.711609,4.099106,-7.887092,0.000651,-0.002079,18.776413,0.232187,0.378686,0.389128,0.665541,0.183354,0.151106,0.502457,0.497543
std,14.955627,349.088352,3.313026,1.112978,3.435737,1.172488,1300.548412,144.157777,379.648948,0.880005,0.876609,10.826849,0.422292,0.485134,0.487627,0.471874,0.387016,0.358207,0.500071,0.500071
min,1.000000,-9600.000000,2007.000000,1.000000,1.000000,0.000000,10389.000000,-3000.000000,-9594.440000,-1.788854,-1.788854,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,-7.300000,2010.000000,1.000000,3.000000,2.000000,11661.500000,-0.520000,-7.500000,-0.670820,-0.662621,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,2013.000000,2.000000,6.000000,3.000000,13059.500000,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,39.000000,5.675000,2015.000000,3.000000,9.000000,3.000000,13697.250000,0.272500,7.280000,0.662280,0.637370,29.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,52.000000,3500.000000,2018.000000,4.000000,12.000000,4.000000,15976.000000,5700.000000,5787.270000,1.788854,1.788854,33.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Split the data in train and test

In [ ]:
x = df_high

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)